# **HitlerGPT**



###- **Requirements**

In [ ]:
!nvidia-smi

Wed Jul 12 21:55:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

###- **Basic settings  & Load datasets**

In [ ]:
outfolder = "model"

newtrain = 1 #@param {type:"string"}

checkpoint = 360000
resumep = f"./{outfolder}/checkpoint-{checkpoint}"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/hitlergpt/ /content/hitlergpt/
%cd hitlergpt

Mounted at /content/drive
/content/hitlergpt


In [ ]:
from datasets import load_dataset
%cd dataset
data = load_dataset('json', data_files='hitlerbook.json')
#data = load_dataset('json', data_files='tqna.json')
%cd ..

[Errno 2] No such file or directory: 'dataset'
/content/hitlergpt/dataset


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8534d57c972be513/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

/content/hitlergpt


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 25857
    })
})

In [ ]:
maxse = 2*int(data.num_rows.get('train'))
print(maxse)

###- **Base model (pythia)**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/pythia-410m-deduped"
#model_id = "EleutherAI/pythia-160m-deduped"
#model_id = "EleutherAI/pythia-70m-deduped"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

###- **Preprocess**

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/25857 [00:00<?, ? examples/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 786432 || all params: 255125504 || trainable%: 0.3082529922214284


###- **Fine tuning**

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        save_strategy="epoch",
        #save_strategy="steps",
        #save_steps= saveinterval,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=maxse,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir=outfolder,
        #optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

if newtrain == 1:
    trainer.train()
elif newtrain == 0:
    trainer.train(resume_from_checkpoint=resumep)
#trainer.train()
#trainer.train(resume_from_checkpoint=resumep)

model.save_pretrained(f"./{outfolder}")

Step,Training Loss
10,4.120000
20,3.935900
30,3.803700
40,3.631100
50,3.884600
60,3.626400
70,4.051000
80,3.821900
90,3.692500
100,3.940800


###- **Evaluation**

In [ ]:
model.eval()
model.config.use_cache = True

##- **Save checkpoint**

In [ ]:
mddir = "/content/drive/MyDrive/hitlergpt/model/" #@param {type:"string"}

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!mkdir -p "{mddir}"
!zip -r ckpt.zip /content/hitlergpt/model/
!cp -vr ckpt.zip "{mddir}"

  adding: content/hitlergpt/model/ (stored 0%)
  adding: content/hitlergpt/model/runs/ (stored 0%)
  adding: content/hitlergpt/model/runs/Jul12_22-08-21_41c961952836/ (stored 0%)
  adding: content/hitlergpt/model/runs/Jul12_22-08-21_41c961952836/events.out.tfevents.1689199701.41c961952836.259.3 (deflated 63%)
  adding: content/hitlergpt/model/runs/Jul12_22-10-33_41c961952836/ (stored 0%)
  adding: content/hitlergpt/model/runs/Jul12_22-10-33_41c961952836/events.out.tfevents.1689199834.41c961952836.259.4 (deflated 72%)
  adding: content/hitlergpt/model/runs/Jul12_22-07-39_41c961952836/ (stored 0%)
  adding: content/hitlergpt/model/runs/Jul12_22-07-39_41c961952836/events.out.tfevents.1689199659.41c961952836.259.2 (deflated 60%)
  adding: content/hitlergpt/model/runs/Jul12_22-06-48_41c961952836/ (stored 0%)
  adding: content/hitlergpt/model/runs/Jul12_22-06-48_41c961952836/events.out.tfevents.1689199608.41c961952836.259.1 (deflated 60%)
  adding: content/hitlergpt/model/runs/Jul12_22-05-34

# Send to Hugging Face

In [ ]:
from huggingface_hub import login
hf_token = ""#@param {type:"string"}
repository = "https://huggingface.co/FENRlR/HitlerGPT"#@param {type:"string"}
login(token=hf_token)
model.push_to_hub(repository)